In [8]:
CREATE EXTENSION plpythonu;


CREATE OR REPLACE FUNCTION execute_python_script(python_script text)
RETURNS text
LANGUAGE plpythonu
AS $$
# Execute the Python script
exec_result = plpy.execute(python_script, 1)

return exec_result[0]["?column?"]
$$;


Error: function execute_python_script(text) does not exist
LINE 1: SELECT execute_python_script('
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.



In [1]:
import psycopg2

# Replace these with your database connection details
db_host = 'localhost'
db_name = 'TestDataBase'
db_user = 'postgres'
db_password = '2357'

python_script = """
print("Hello from Python script!")
"""

try:
    # Connect to the database
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )

    # Create a cursor
    cursor = conn.cursor()

    # Define the function call
    function_call = "SELECT * from Employees"
# execute_python_script('{python_script}')"

    # Execute the function
    cursor.execute(function_call)

    # Fetch the result
    result = cursor.fetchone()

    # Print the result
    print("Result from PostgreSQL:", result)

    # Close the cursor and the connection
    cursor.close()
    conn.close()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)


Error while connecting to PostgreSQL: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



In [3]:
import openai

import json

import psycopg2

import os

import ast

import re

import time

import datetime

from contextlib import suppress

openai.api_key= '46d7ed565dda46f1893e7d0e42b3aefb'
openai.api_base = 'https://openaiautomationteam.openai.azure.com/'
openai.api_type= 'azure'
TEMPERATURE = 0.7


# STOP = os.getenv('STOP', "")
# MAX_TOKENS = float(os.getenv('MAX_TOKENS', 500))
# TOP_P = int(os.environ.get('TOP_P', 1))
# FREQUENCY_PENALTY = int(os.getenv('FREQUENCY_PENALTY', 0))
# PRESENCE_PENALTY = int(os.getenv('PRESENCE_PENALTY', 0))
# N_RESP = int(os.getenv('N_RESP', 1))
# TIMEOUT = int(os.getenv('TIMEOUT', 60))

DB_HOST = os.environ.get('DB_HOST', None)
DB_PORT = os.environ.get('DB_PORT', None)
DB_USER = os.environ.get('DB_USER', None)
DB_PASSWORD = os.environ.get('DB_PASSWORD', None)
DB_NAME = os.environ.get('DB_NAME', None)

PGRESCREDS={'Host': DB_HOST, 'Port': DB_PORT, 'User': DB_USER, 'Password': DB_PASSWORD, 'Name': DB_NAME}

Creds = list(PGRESCREDS.values())

sql={"sql":"SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public';"}


sql_key = list(sql.keys())

base_user_prompt = open('config/user_prompt.txt', 'r').read()

input_prompt=input("Enter the input prompt: ") #user input

system_prompt = open('config/system_prompt.txt', 'r').read()
system_prompt = system_prompt.replace('<input>', input_prompt)

history = "Empty"
user_prompt = base_user_prompt.replace('<history>', history[0])

current_time = datetime.datetime.now()
current_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
user_prompt = user_prompt.replace('<DateTime>', current_time)

In [8]:
arguments = dict(
        engine='DIR_GPT4',
        temperature=0.7,
        messages=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]
    )

result_GPT = openai.ChatCompletion.create(**arguments)
response_message = result_GPT.choices[0].message.content
print(content)
    

Thought: Let's start by understanding the database, tables, columns, descriptions, and datatypes.

```
{
  "function": "ExecuteSQL",
  "parameters": {
    "sql": "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'"
  }
}
```


In [6]:

def automate_function_call(Creds, sql_key, response_message):

    """Function calling for Running the generated python code on PostGRES Server"""


    conn = psycopg2.connect(host=Creds[0], database=Creds[4], user=Creds[2], password=Creds[3], port=Creds[1])
    
    def extract_outermost_dict(s):
        for i in range(len(s)):
            for j in range(i, len(s)):
                substring = s[i:j+1]
                try:
                    node = ast.literal_eval(substring)
                    if isinstance(node, dict):
                        return node
                except (ValueError, SyntaxError):
                    pass
        return None
    
    def extract_json(response_message):
        """Function to extract JSON from the output response. The JSON is within triple bacticks

        Args:
            output_response (str): output response

        Returns:
            dict: json
        """
        json_output = re.findall(r"```([\s\S]*?)```", response_message)

        if len(json_output) == 0:
            return ''
        elif len(json_output) >= 1:
            json_output = json_output[0]
        
        json_output = extract_outermost_dict(json_output)
        return json_output
    
    def validate_json(self, input_json):
        return True, "The JSON is valid"

    json_output = extract_json(response_message)

    def ExecuteSQL(sql_key):
        """Function to execute SQL query

        Args:
            sql (str): sql query to execute

        Returns:
            str: output of sql query
        """
        try:
            sql = sql[sql_key[0]] #key
            cur = conn.cursor()
            # Execute a SQL query
            cur.execute(sql)

            # Fetch the results
            results = cur.fetchall()

            # Close the cursor and connection
            cur.close()
            
            if results == '':
                return "PostgresSQL Query executed Successfully"
            else:
                ShareOutput(results)
                return results
        except Exception as e:
            return f"There is some error in SQL query: {str(e)}"
    
    def get_database_info(sql_key):
        """Function to get database and table information using sql queries

        Returns:
            dict: database info
        """
        try:
            result = ExecuteSQL(sql_key)
            update_history({"function":"ExecuteSQL","parameters":{"sql":"SELECT table_name, column_name, data_type FROM information_schema.columns WHERE table_schema = 'public';"}}, result)
        except Exception as e:
            raise e

    def ShareOutput(output: str) -> str:
        """Function to share output

        Args:
            output (str): output to share

        Returns:
            str: output of sharing output
        """
        try:
            return output
        except Exception as e:
            return e

    def update_history(input_json: str, output: str) -> str:
        """
        Function to update history

        Args:
            input_prompt (str): input prompt
            output (str): output

        Returns:
            str: output of updating history
        """
        try:
            history = prepare_history(input_json, output)
            if history == "Empty":
                history = [history]
            else:
                history.append(history)
            
            for ind, his in enumerate(history, start=1):
                history += f"{ind}. {his}\n"

            user_prompt = base_user_prompt.replace('<history>', f"[{history}]")
        except Exception as e:
            raise e
        
    def prepare_history(input_json: str, output: str) -> str:
        """Function to prepare history

        Args:
            input_prompt (str): input prompt
            output (str): output

        Returns:
            str: output of preparing history
        """
        try:
            return f"Assistant Response-\n{input_json}\nExecution Output-\n{output}\n"
        except Exception as e:
            raise e

In [9]:
automate_function_call(Creds, sql_key, response_message)

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
